# 线性回归 --- 使用 Gluon

[前一章]我们仅仅使用了 `ndarray` 和 `autograd` 来实现线性回归，这一章我们仍然实现同样的模型，但是使用高层抽象包 `gluon`。

## 创建数据集

我们生成同样的数据集

In [3]:
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet import gluon

num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

X = nd.random_normal(shape=(num_examples, num_inputs))
y = true_w[0] * X[:, 0] + true_w[1] * X[:, 1] + true_b
y += .01 * nd.random_normal(shape=y.shape)

## 数据读取

但这里使用 `data` 模块来读取数据。

In [4]:
batch_size = 10
dataset = gluon.data.ArrayDataset(X, y)
data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle=True)

读取跟前面一致：

In [5]:
for data, label in data_iter:
    print(data, label)
    break


[[-2.00694501e-02 -2.32134894e-01]
 [-5.51408768e-01  4.12447691e-01]
 [ 5.94287276e-01  1.86423331e-01]
 [ 1.23247862e+00  4.42714244e-01]
 [ 1.22368865e-01  2.20038489e-01]
 [ 2.95364201e-01  2.84753174e-01]
 [ 2.67223287e+00  2.04891384e-01]
 [ 1.22062273e-01 -1.41378367e+00]
 [-2.59043509e-03 -8.19595903e-02]
 [ 1.05900943e+00  1.82906449e+00]]
<NDArray 10x2 @cpu(0)> 
[4.9358025  1.7047348  4.753762   5.158114   3.6787946  3.8277013
 8.845208   9.25077    4.477494   0.10042815]
<NDArray 10 @cpu(0)>


## 定义模型

之前一章中，当我们从0开始训练模型时，需要先声明模型参数，然后再使用它们来构建模型。但 `gluon` 提供大量预定义的层，我们只需要关注使用哪些层来构建模型。例如线性模型就是使用对应的 `Dense` 层；之所以称为 dense层，是因为输入的所有节点都与后续的节点相连。在这个例子中仅有一个输出，但在大多数后续章节中，我们会用到具有多个输出的网络。

我们之后还会介绍如何构造任意结构的神经网络，但对于初学者来说，构建模型最简单的办法是利用 `Sequential` 来所有层串起来。输入数据之后，`Sequential` 会依次执行每一层，并将前一层的输出，作为输入提供给后面的层。首先我们定义一个空的模型：


In [6]:
net = gluon.nn.Sequential()

然后我们加入一个 `Dense` 层，它唯一必须定义的参数就是输出节点的个数，在线性模型里面是 $1$.

In [8]:
net.add(gluon.nn.Dense(1))

**注意**：这里我们并没有定义说这个层的输入节点是多少，这个在之后真正给数据的时候系统会自动赋值。我们之后会详细介绍这个特性是如何工作的。

## 初始化模型参数

在使用前`net`我们必须要初始化模型权重，这里我们使用默认随机初始化方法（之后我们会介绍更多的初始化方法）。

In [9]:
net.initialize()

## 损失函数

`gluon`提供了平方误差函数：

In [10]:
square_loss = gluon.loss.L2Loss()

## 优化

同样我们无需手动实现随机梯度下降，我们可以用创建一个`Trainer`的实例，并且将模型参数传递给它就行。

In [11]:
trainer = gluon.Trainer(
    net.collect_params(), 'sgd', {'learning_rate': 0.1})

## 训练
使用`gluon`使模型训练过程更为简洁。我们不需要挨个定义相关参数、损失函数，也不需使用随机梯度下降。`gluon`的抽象和便利的优势将随着我们着手处理更多复杂模型的愈发显现。不过在完成初始设置后，训练过程本身和前面没有太多区别，唯一的不同在于我们不再是调用`SGD`，而是`trainer.step`来更新模型（此处一并省略之前绘制损失变化的折线图和散点图的过程，有兴趣的同学可以自行尝试）。

In [12]:
epochs = 5
batch_size = 10
for e in range(epochs):
    total_loss = 0
    for data, label in data_iter:
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        total_loss += nd.sum(loss).asscalar()
    print("Epoch %d, average loss: %f" % (e, total_loss/num_examples))

Epoch 0, average loss: 0.932457
Epoch 1, average loss: 0.000046
Epoch 2, average loss: 0.000047
Epoch 3, average loss: 0.000046
Epoch 4, average loss: 0.000047


比较学到的和真实模型。我们先从`net`拿到需要的层，然后访问其权重和位移。

In [13]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 1.9997537 -3.3999002]]
 <NDArray 1x2 @cpu(0)>)

In [14]:
true_b, dense.bias.data()

(4.2, 
 [4.2010345]
 <NDArray 1 @cpu(0)>)

## 结论

可以看到 `gluon` 可以帮助我们更快更干净地实现模型。


## 练习

- 在训练的时候，为什么我们用了比前面要大10倍的学习率呢？（提示：可以尝试运行 `help(trainer.step)`来寻找答案。）
- 如何拿到`weight`的梯度呢？（提示：尝试 `help(dense.weight)`）

**吐槽和讨论欢迎点**[这里](https://discuss.gluon.ai/t/topic/742)